In [1]:
from pandas import pandas as pd
import numpy as np
import csv, sqlite3
import logging
import cProfile
import mplib as mp

In [2]:
number_of_test_subjects = 4
lz = mp.lazy_pandas()
client_info = lz.read_csv('../MOCK_DATA/mock_data_1.csv')
male_clients = client_info['gender'] == 'Male'
test_male_client = client_info[ male_clients ][:number_of_test_subjects]
print(test_male_client)

print(client_info is male_clients)

over read_csv () {}
over con () {}
Creating engine
caught []
caught eq
caught []
caught []
 lazy Df <sqlite3.Connection object at 0x7fae14a2c8f0>
project gender 
predicate equal Male 
project df 
project slice(None, 4, None) 


True


In [7]:
number_of_test_subjects = 8
## pandas
with mp.Profiler(2):
    client_info = pd.read_csv('../MOCK_DATA/mock_data_1.csv')
    male_clients = client_info['gender'] == 'Male'
    test_male_client = client_info[ male_clients ][:number_of_test_subjects]
    pd_out = test_male_client.ip_address

## sqlite3
with mp.Profiler(2):
    con = sqlite3.connect(':memory:') # or file or ?
    db = mp.csvToDb('../MOCK_DATA/mock_data_1.csv', con=con, table_name='client_info')
    query = """
    select ip_address
    from client_info
    where gender = 'Male'
    limit """ + str(number_of_test_subjects) + """
    """
    sql_out = db.execute(query)
    sql_label = [description[0] for description in sql_out.description]
    sql_data = sql_out.fetchall()

## now check
for i, (sql_x, pd_x) in enumerate(zip(sql_data, pd_out)):
    if i < number_of_test_subjects//2:
        print(sql_x[0], '|', pd_x)
    assert sql_x[0] == pd_x
print('     PASSED')

0     False
1      True
2     False
3      True
4      True
5     False
6      True
7      True
8      True
9     False
10    False
11    False
12    False
13     True
14    False
15     True
16    False
17     True
18     True
19     True
20     True
21    False
22     True
23    False
24    False
25    False
26    False
27    False
28    False
29     True
      ...  
70     True
71     True
72    False
73     True
74     True
75    False
76    False
77    False
78    False
79    False
80    False
81    False
82    False
83     True
84    False
85     True
86    False
87     True
88     True
89    False
90    False
91     True
92    False
93    False
94    False
95     True
96    False
97     True
98    False
99     True
Name: gender, Length: 100, dtype: bool
         6961 function calls (6915 primitive calls) in 0.011 seconds

   Ordered by: cumulative time
   List reduced from 494 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  

In [4]:
client_info = pd.read_csv('../MOCK_DATA/mock_data_1.csv')
works_at = pd.read_csv('../MOCK_DATA/mock_data_2.csv')

# dumb pandas
print('email company name')
print('------------------')
res = []
with mp.Profiler(2):
    for id_w, w in enumerate(works_at.email):
        for id_c, c in enumerate(client_info.email):
            if w == c:
                res.append((works_at.email[id_w], works_at.company[id_w], client_info.first_name[id_c]))
print(res)

email company name
------------------
         160777 function calls (160773 primitive calls) in 0.104 seconds

   Ordered by: cumulative time
   List reduced from 89 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2001    0.003    0.000    0.077    0.000 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:1049(__iter__)
     2001    0.004    0.000    0.070    0.000 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/dtypes/common.py:609(is_datetimelike)



[('keble29@usatoday.com', 'Wordware', 'Karly'), ('dkobud@google.es', 'Wikivu', 'Durand'), ('keble29@usatoday.com', 'Dabshots', 'Karly')]


In [5]:
## pandas
print('email company name')
print('------------------')
res = []
with mp.Profiler(2):
    matches = works_at.email.isin(client_info.email)
    for email, company in zip( works_at[matches].email, works_at[matches].company):
        guy = client_info[client_info.email == email]
        res.append([email, company, guy.first_name.values[0]])
print(res)


email company name
------------------
         4167 function calls (4099 primitive calls) in 0.009 seconds

   Ordered by: cumulative time
   List reduced from 257 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       15    0.000    0.000    0.006    0.000 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:1940(__getitem__)
        5    0.000    0.000    0.005    0.001 /home/marcello/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:1983(_getitem_array)



[['keble29@usatoday.com', 'Wordware', 'Karly'], ['dkobud@google.es', 'Wikivu', 'Durand'], ['keble29@usatoday.com', 'Dabshots', 'Karly']]


In [6]:
## sql
con = sqlite3.connect(':memory:') # or file or ?
a = mp.csvToDb('../MOCK_DATA/mock_data_1.csv', con=con, table_name='client_info')
b = mp.csvToDb('../MOCK_DATA/mock_data_2.csv', con=con, table_name='works_at')
with mp.Profiler(2):
    query = """
    select works_at.email, works_at.company, client_info.last_name
    from client_info, works_at
    where client_info.email = works_at.email
    """
    sql_out = con.execute(query)
    sql_label = [description[0] for description in sql_out.description]
    sql_data = sql_out.fetchall()
print(sql_data)

         5 function calls in 0.003 seconds

   Ordered by: cumulative time
   List reduced from 5 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 {method 'execute' of 'sqlite3.Connection' objects}
        1    0.000    0.000    0.000    0.000 {method 'fetchall' of 'sqlite3.Cursor' objects}



[('email', 'company', 'last_name'), ('dkobud@google.es', 'Wikivu', 'Kobu'), ('keble29@usatoday.com', 'Dabshots', 'Eble'), ('keble29@usatoday.com', 'Wordware', 'Eble')]
